**라이브러리 로드**

In [ ]:
import torch
print("torch.cuda.is_available() =", torch.cuda.is_available())
print("torch.cuda.device_count() =", torch.cuda.device_count())
print("torch.version.cuda =", torch.version.cuda)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

***라이브러리 설치 - RoboFlow***

In [ ]:
!pip install roboflow ultralytics -q

**Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

**Roboflow**

In [ ]:
# ==========================================
# Roboflow 데이터 다운로드
# ==========================================

from roboflow import Roboflow

ver_num = 9

rf = Roboflow(api_key="   ")
project = rf.workspace("yunhoo93").project("dataset-bm9aj")
dataset = project.version(ver_num).download("yolov8")

**YOLO Fine-tuning**

In [ ]:
from ultralytics import YOLO
import shutil, os

class DriveBackupCallback:
    def __init__(self, drive_dir="/content/drive/MyDrive/yolo_epoch_backups", interval=5):
        self.drive_dir = drive_dir
        self.interval = interval
        os.makedirs(drive_dir, exist_ok=True)

    def on_train_epoch_end(self, trainer):
        epoch = trainer.epoch
        # 🔹 매 5 epoch마다 백업 수행
        if (epoch + 1) % self.interval == 0:
            src = trainer.save_dir
            dst = f"{self.drive_dir}/ver{ver_num}_epoch_{epoch+1}"
            if not os.path.exists(dst):
                shutil.copytree(src, dst)
            print(f"✅ Drive 백업 완료: {dst}")
        else:
            print(f"⏭️ Epoch {epoch+1}: 백업 건너뜀 (interval={self.interval})")


# -----------------------------------------------------------
#  📌 YOLO11x 모델 로드
# -----------------------------------------------------------
#  - 대규모 객체(도로, 펜스, 가로등, 차량)를 다루기 위한 고성능 백본
#  - 파라미터 수가 많아 세밀한 특징 추출 가능
model = YOLO("yolo11x.pt")

# 5 epoch마다 백업
model.add_callback("on_train_epoch_end", DriveBackupCallback(interval=5).on_train_epoch_end)


model.train(
    data=f"{dataset.location}/data.yaml",


    # ==========================================================
    #   1) Generalization (데이터 다양성 확보 + 기본 학습 안정화)
    # ==========================================================
    epochs=300,           # 🔼 다양한 환경(도로/야간/각도)을 커버하려고 충분한 epoch
    patience=40,          # 🔼 과적합 방지: 개선이 없으면 조기 종료
    imgsz=1024,           # 🔼 대형 모델과 도로 객체 검출에 적합한 해상도
    batch=8,
    optimizer="AdamW",    # 🔼 일반화 성능 우수한 옵티마이저

    # LR/Decay (Generalization 핵심)
    lr0=0.00028,          # 안정적인 초기 학습률
    lrf=0.1,              # 점진적으로 감소하도록 설정
    weight_decay=0.00065, # 과적합 방지 + 일반화 증가

    # Warmup (초기 수렴 안정화)
    warmup_epochs=5,
    warmup_momentum=0.9,

    # 데이터 증강 → 다양성(Generalization)
    hsv_h=0.015,
    hsv_s=0.35,
    hsv_v=0.35,
    mosaic=0.10,          # 도로/배경 다양화
    translate=0.05,
    scale=0.20,
    shear=0.0,
    perspective=0.0005,
    flipud=0.0,
    fliplr=0.4,           # 차량/도로 좌우 반전은 매우 효과적
    mixup=0.1,            # damage_road 노이즈 대응
    copy_paste=0.1,       # 펜스/가로등 같은 점형 객체에 유효
    erasing=0.3,          # texture 보강
    auto_augment="randaugment",


    # ==========================================================
    #   2) Precise Box (박스 정밀도 향상 중심)
    # ==========================================================
    box=7.0,              # 박스 회귀 loss 가중치 증가
    cls=0.8,              # 클래스 분리 강화 → 불균형 클래스 대응
    kobj=1.0,
    label_smoothing=0.03, # 과적합 완화
    dropout=0.05,         # 작은 객체 안정화

    # close-mosaic → 후반에서는 mosaic OFF (정확도 집중)
    close_mosaic=20,      # 🔥 Precise Box 단계에 해당


    # ==========================================================
    #   3) Ultra Fine (후반부 초정밀 최적화 단계)
    # ==========================================================
    cos_lr=True,          # cosine learning rate → 마지막까지 부드럽게 수렴


    # ==========================================================
    #   기타 설정
    # ==========================================================
    amp=True,
    workers=8,
    project="train_results",
    name="yolo11x_final",
    save_json=True,
    save_hybrid=True,
    resume=False,
    seed=42,
    plots=True,
)